<a href="https://colab.research.google.com/github/NewCodeLearner/Build_DeepSeek_from_scratch/blob/main/01_multi_head_latent_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we code the Multi-Head Latent Attention (MLA) Mechanism from scratch in Python.

MLA was one of the key innovations in the DeepSeek architecture.

Here, we code the simplest variant of MLA without rotary positional encodings added.

# Step 1: Writing the code for the Multi-Head Latent Attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RopelessMLA(nn.Module):
  def __init__(self,d_model,n_heads,kv_latent_dim):
    super().__init__()
    self.d_model = d_model
    self.n_heads = n_heads
    self.dh = d_model // n_heads # dimension per head

    #Projection layers
    self.W_q = nn.Linear(d_model,d_model, bias = False)            # Query Projection
    self.W_dkv = nn.Linear(d_model,kv_latent_dim, bias = False)    # Compress into latent KV space
    self.W_uk = nn.Linear(kv_latent_dim, d_model, bias = False)    # Decompress K
    self.W_uv = nn.Linear(kv_latent_dim,d_model, bias = False)     # Decompress V
    self.W_o = nn.Linear(d_model,d_model, bias = False)            # Final output projection

    self.ln = nn.LayerNorm(kv_latent_dim)
    self.register_buffer('absorbed_k', None)  #Holds W_q @ W_uk
